<a href="https://colab.research.google.com/github/Jigisha-Pradhan/Smart-Lock-Face-Recognition-System/blob/main/Face_Detection_based_door_lock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip known_faces

unzip:  cannot find or open known_faces, known_faces.zip or known_faces.ZIP.


In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=55cc3542fa94cccb3ced0ad079259c6c7925a9e91cc29db2363d7bfbed056872
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from deepface import DeepFace
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from collections import Counter

In [ ]:
# Assuming your images are in a folder named 'known_faces'
dataset = tf.keras.utils.image_dataset_from_directory(
    'known_faces',
    labels='inferred',  # Infer labels from directory structure
    label_mode='int',   # Integer encoded labels
    image_size=(256, 256), # Resize images to 256x256
    batch_size=32,      # Number of images per batch
    shuffle=True        # Shuffle the dataset
)

Found 12 files belonging to 4 classes.


In [ ]:
def load_dataset(dataset_path):
    """
    Loads a dataset of images from the given path and extracts facial embeddings.

    Args:
        dataset_path (str): Path to the dataset directory.

    Returns:
        tuple: A tuple containing:
            - np.ndarray: Array of facial embeddings (X).
            - np.ndarray: Array of corresponding labels (y).
            Returns ([],[]) if no data found.
    """
    X, y = [], []
    labels = os.listdir(dataset_path)

    for label in labels:
        person_path = os.path.join(dataset_path, label)
        if os.path.isdir(person_path):
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                try:
                    embedding = DeepFace.represent(img_path, model_name="Facenet512", enforce_detection=True)[0]['embedding']
                    X.append(embedding)
                    y.append(label)
                except:
                    print(f"Skipping {img_path}, face not detected.")
    if not X:
        print(f"No data found in directory: {dataset_path}")
        return [], [] # Return empty lists as a tuple.
    return np.array(X), np.array(y)

def train_model(X, y):
    """
    Trains a deep learning model on the given dataset.

    Args:
        X (np.ndarray): Array of features.
        y (np.ndarray): Array of labels.

    Returns:
        tuple: A tuple containing the trained model, scaler, and label_to_index mapping.
        Raises Exception if no data found.
    """
    if not X.size or not y.size:
        raise ValueError("No data provided for training. Check load_dataset output.")

    if len(set(y)) < 2:
        print("Not enough classes to train. Add more images per person.")
        raise ValueError("Not enough classes to train. Add more images per person.")

    print(f"Training on {len(X)} samples across {len(set(y))} identities.")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if len(set(y_train)) < 2:
        print("Not enough classes in the training set. Increase dataset size.")
        raise ValueError("Not enough classes in the training set. Increase dataset size.")

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Handle class imbalance
    class_counts = Counter(y_train)
    min_samples = min(class_counts.values())

    if min_samples > 1:
        k = min(2, min_samples - 1)
        smote = SMOTE(k_neighbors=k)
        X_train, y_train = smote.fit_resample(X_train, y_train)
    else:
        print("Skipping SMOTE due to insufficient samples per class.")

    # Convert labels to categorical
    unique_labels = list(set(y))
    label_to_index = {label: i for i, label in enumerate(unique_labels)}
    y_train = np.array([label_to_index[label] for label in y_train])
    y_test = np.array([label_to_index[label] for label in y_test])
    y_train = to_categorical(y_train, num_classes=len(unique_labels))
    y_test = to_categorical(y_test, num_classes=len(unique_labels))

    # Define a deep learning model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Deep Learning Model Accuracy: {test_acc * 100:.2f}%")

    return model, scaler, label_to_index

def verify_face(model, scaler, label_to_index, img_path):
    """
    Verifies the identity of a face in an image.

    Args:
        model: The trained deep learning model.
        scaler: The scaler used for feature normalization.
        label_to_index: The mapping from label to index.
        img_path (str): The path to the image.

    Returns:
        str: The predicted identity.
        None: If not recognised.
    """
    embedding = DeepFace.represent(img_path, model_name="Facenet512", enforce_detection=True)[0]['embedding']
    embedding = scaler.transform([embedding])
    prediction = model.predict(embedding)
    predicted_label = list(label_to_index.keys())[np.argmax(prediction)]
    confidence = np.max(prediction) * 100
    if(confidence>=70):
        print(f"Predicted Identity: {predicted_label}, Confidence: {confidence:.2f}%")
        return predicted_label
    else:
        print("Not recognised")
        return None

In [ ]:
dataset_path = "/content/known_faces"
X, y = load_dataset(dataset_path)
#Check if no data.
if X.size == 0: # Check if the array is empty using .size
    print("Exiting. No valid data found.")
else:
    try:
        model, scaler, label_to_index = train_model(X, y)
    except ValueError as e:
        print(f"Failed to train: {e}")

        # Test the model
        # Ensure model is trained before testing
    if 'model' in locals():
        test_img = "TestJ1.jpg"
        predicted_identity = verify_face(model, scaler, label_to_index, test_img)

25-03-29 13:59:49 - facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:00<00:00, 161MB/s]


Skipping /content/known_faces/Bindu/bindhu2.jpg, face not detected.
Training on 11 samples across 4 identities.
Skipping SMOTE due to insufficient samples per class.
Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 2.3699 - accuracy: 0.2500 - val_loss: 1.2583 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 52ms/step - loss: 1.1525 - accuracy: 0.6250 - val_loss: 0.8273 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 35ms/step - loss: 0.6601 - accuracy: 0.7500 - val_loss: 0.5531 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 35ms/step - loss: 0.4243 - accuracy: 0.8750 - val_loss: 0.3971 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 54ms/step - loss: 0.3907 - accuracy: 0.8750 - val_loss: 0.2865 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 53ms/step - loss: 0.2997 - accuracy: 0.8750 - val_loss: 0.2078 - val_accuracy: 1.000